In [2]:
!pip install -q tritonclient[http]
!pip install -q pandas scikit-learn

In [3]:
import tritonclient.http as httpclient
from tritonclient.utils import np_to_triton_dtype

In [4]:
import pandas as pd
import numpy as np
import json

In [5]:
client = httpclient.InferenceServerClient(url='localhost:8000')

In [6]:
data_infer = pd.read_csv("data_infer.csv").astype('str')

In [7]:
input_data = data_infer.iloc[:4].values

In [8]:
input_data

array([['1005', '0', '2013', '6', '5', '22:03', '$-180.00',
        'Swipe Transaction', '7834055923142137930', 'Orlando', 'FL',
        '32808.0', '3395', 'nan'],
       ['134', '5', '2015', '2', '16', '18:34', '$99.10',
        'Chip Transaction', '-1548923525906069124', 'Tyler', 'TX',
        '75706.0', '4900', 'nan'],
       ['1270', '0', '2009', '11', '27', '18:44', '$82.42',
        'Swipe Transaction', '-7329090674102508598', 'Saint Charles',
        'IL', '60174.0', '5812', 'nan'],
       ['109', '0', '2010', '11', '3', '16:21', '$0.45',
        'Swipe Transaction', '-6571010470072147219', 'Dyer', 'IN',
        '46311.0', '5499', 'nan']], dtype=object)

In [9]:
def prepare_tensor(name, input_data):
    tensor = httpclient.InferInput(
        name, input_data.shape, np_to_triton_dtype(input_data.dtype))
    tensor.set_data_from_numpy(input_data)
    return tensor

In [24]:
input_data.dtype

dtype('O')

In [23]:
np_to_triton_dtype(input_data.dtype)

'BYTES'

In [10]:
model_name = "ensemble_preprocess_fil_postprocess"

In [11]:
inputs = [prepare_tensor('raw_data', input_data)]

In [12]:
input_data.shape

(4, 14)

In [ ]:
(4, )
(bs, )

In [14]:
np.array([0, 1, 0, 0]).shape

(4,)

In [ ]:
(-1, 1), (4, 1)

In [19]:
response = client.infer(model_name,
                        inputs)

In [20]:
output_data = response.as_numpy("processed_predictions")

In [26]:
output_data

array([b'NOT FRAUD', b'NOT FRAUD', b'NOT FRAUD', b'NOT FRAUD'],
      dtype=object)

In [21]:
output_data.shape

(4,)

In [24]:
output_data

array([0., 0., 0., 0.], dtype=float32)

In [15]:
output_data.shape

(4, 2)

In [16]:
data = pd.DataFrame(['nan'], columns=['Errors?']).astype(str)

In [13]:
data['Errors?'] != 'nan'

0    False
Name: Errors?, dtype: bool

In [86]:
print(output_data)

[45.23]


In [105]:
CLASS_LABELS = ['NOT FRAUD', 'FRAUD']

In [106]:
class_indices = np.array([1.0, 0.0, 1.1, 1.1])
map_func = lambda idx : CLASS_LABELS[int(idx)]
class_labels = map_func(class_indices)

TypeError: only size-1 arrays can be converted to Python scalars